# Creating a Funding Transaction

In this section we'll create a Lightning Channel funding transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A high level understanding of the Lightning Network. e.g. [Mastering Lightning](https://github.com/bitcoinbook/bitcoinbook) TODO Get the chapter to point it here
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).

- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'

## Channel Establishment v1

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/00-introduction.md)) defines two pathways to create a channel, lets focus on the legacy one because most of the channels are created using it, and lets the 'Channel Establishment v2' for the following notebook.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'



Alice send the [`open_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-open_channel-message) message to Bob. This message define many channel operational criterias, here we are going to focus on the information that are used in the funding transaction:
* chain_hash - denotes the exact blockchain that the opened channel will reside within.
* funding_satoshis - is the amount the sender is putting into the channel
* funding_pubkey - is the public key in the 2-of-2 multisig script of the funding transaction output.


If Bob does not agree with the criterias sent into the `open_channel` message he send a [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice, otherwise he sends the [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, containing some operational criterias from your side, here we are going to focus on the information that are used in the funding transaction:
* funding_pubkey - is the public key in the 2-of-2 multisig script of the funding transaction output.

## The Funding Transaction

The funding transaction input could be one or more UTXOs from Alice (the funder) and the output arethea change to Alice and a P2WSH output (pay-to-witness-script-hash)<sup>[BIP141](https://github.com/bitcoin/bips/blob/master/bip-0141.mediawiki#witness-program)</sup> funding the channel, the order of the pubkeys on the output script is defined lexicographically, where pubkey1 is the lesser of the two funding_pubkey in compressed format and pubkey2 is the greater.

`2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG`

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing lightning-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/home/pins-dev/Projects/lightning-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the lightning-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

In [2]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 0 - Key Derivation/The Lightning Node Transactions Keys.ipynb"

Alice Funding Private Key: 23c334147ed9438c41de42a848186cf66a1fc7ac9b9188b7bb048784d3d8957f
Alice Funding Public Key: 028e3fe2292fbcf3d5dbe3ae8d6589022e5188e8e3045c2c817a8ef9f643246b0d
Alice Change Private Key: 016ed3c51e103c3af4c7881c39e193058f98604e61838dc328747ce62f93476c
Alice Change Public Key: 035cf94ac57d02edc7ea0cb466ad3e91d758a6a8926ea759d23e180cdc54a2edf5
Bob Private Key: 58a2d30fbb47919a9e06a67d296c5623fdb81d8b6602fa17b32d2737f1a6a379
Bob Public Key: 038fdb901132874d1c7722c0c2074010216655af5d6019fab965d5ffba8724a931


### Create a P2WPKH UTXO

In order to have Alice as funder of a channel we are going to create P2WPKH UTXO. To do that, we'll define a private key for Alice node and use it to generate a pubkey, then convert the pubkey to a p2wpkh address, we do the same process to create a change address (when you are running a lightning node, you would use wallet commands to derive addressess to use).

In [3]:
alice_funding_address = pk_to_p2wpkh(bytes.fromhex(alice_funding_pubkey), network = "regtest")
print("Alice's funding p2wpkh address: " + alice_funding_address)
alice_change_address = pk_to_p2wpkh(bytes.fromhex(alice_change_pubkey), network = "regtest")
print("Alice's change p2wpkh address: " + alice_change_address)

Alice's funding p2wpkh address: bcrt1qeq3ajrptxlst44rpvcfdqyhduqgu9lvz4gvv8z
Alice's change p2wpkh address: bcrt1qp3mznwn4pth86kpsehyvjs564c0smlaxfzt480


 Finally we send 0.201 BTC to the alice funding address.

In [4]:
node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, alice_funding_address, 0.201)
print(f"txid_to_spent: {txid_to_spend}, {index_to_spend}")

2025-01-15T17:32:28.158000Z TestFramework (INFO): PRNG seed is: 9185327270521779376
2025-01-15T17:32:28.160000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_lf5vrs49
txid_to_spent: fdb6e556fecaf24b5ad6f5a40e84b500adebeb5028b646b698a6a0b6bb0b11ce, 0


At this moment Alice would send Bob the `open_channel` message and Bob would answer with `accept_channel` message containing its pubkey.So lets' create below the Bob's pubkey.

### Create the Channel P2WSH multisig script

In order to create the channel funding transaction, we are going to create the 2 of 2 multisig script.

For more on this step, review the 'Bitcoin Script' notebook.

In [5]:
# Compare the public keys lexicographically
if alice_funding_pubkey < bob_funding_pubkey:
    print(f"Alice pubkey is lexicographically lesser than Bob pubkey:\n{alice_funding_pubkey} < {bob_funding_pubkey}")
    pubkey1 = alice_funding_pubkey
    pubkey2 = bob_funding_pubkey
elif alice_funding_pubkey > bob_funding_pubkey:
    print(f"Alice pubkey is lexicographically greater than Bob pubkey:\n{alice_funding_pubkey} > {bob_funding_pubkey}")
    pubkey2 = alice_funding_pubkey
    pubkey1 = bob_funding_pubkey
else:
    print("Error: the public keys are equal.")

# Redeem Script: 2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG
# "21" is the length of a 33 bytes (compressed) pubkey in hex notation
redeemScript = bytes.fromhex(
    "52" 
    + "21"
    + pubkey1
    + "21"
    + pubkey2
    + "52"
    + "ae")

print(f"Redeem Script:\n{redeemScript.hex()}")

Alice pubkey is lexicographically lesser than Bob pubkey:
028e3fe2292fbcf3d5dbe3ae8d6589022e5188e8e3045c2c817a8ef9f643246b0d < 038fdb901132874d1c7722c0c2074010216655af5d6019fab965d5ffba8724a931
Redeem Script:
5221028e3fe2292fbcf3d5dbe3ae8d6589022e5188e8e3045c2c817a8ef9f643246b0d21038fdb901132874d1c7722c0c2074010216655af5d6019fab965d5ffba8724a93152ae


### Convert the Redeem Script to a P2WSH address

For more on this step, review the 'Addresses' notebook.

In [6]:
channel_address = script_to_p2wsh(redeemScript, "regtest")
print("channel_address: ",channel_address)

channel_address:  bcrt1qt85jxm39pf3mkcjm6jfuafcfpfmycc3x2ak85ru348easj5s0hgsy00smf


### Create an unsigned channel funding transaction

The first thing we'll do is define the inputs and outputs of our funding transaction.

In [8]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `alice_funding_privkey` and `alice_funding_pubkey`

# Our outputs
# Determine our output scriptPubkeys and amounts (in satoshis)
# Output1 is the Alice Change P2WPKH
output1_value_sat = int(float("0.15") * 100000000)
pk_hash = hash160(bytes.fromhex(alice_change_pubkey))
output1_spk = bytes.fromhex("0014") + pk_hash
# Output2 is the Channel P2WSH
output2_value_sat = int(float("0.05") * 100000000)
script_hash = hashlib.sha256(redeemScript).digest()
output2_spk = bytes.fromhex("0020") + script_hash
# The Alice UTXO was 0.201, we are leaving 0.001 as the fee of the funding trransaction

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty.

In [9]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_tx: ", unsigned_tx.hex())



unsigned_tx:  0200000001ce110bbbb6a0a698b646b62850ebebad00b5840ea4f5d65a4bf2cafe56e5b6fd0000000000ffffffff02c0e1e400000000001600140c7629ba750aee7d5830cdc8c9429aae1f0dffa6404b4c000000000022002059e9236e250a63bb625bd493cea7090a764c6226576c7a0f91a9f3d84a907dd100000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [10]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "2f72dcb63ff1f44b02f0f27218f68b694787d2dc8fd8a16ced438be850c9264e",
  "hash": "2f72dcb63ff1f44b02f0f27218f68b694787d2dc8fd8a16ced438be850c9264e",
  "version": 2,
  "size": 125,
  "vsize": 125,
  "weight": 500,
  "locktime": 0,
  "vin": [
    {
      "txid": "fdb6e556fecaf24b5ad6f5a40e84b500adebeb5028b646b698a6a0b6bb0b11ce",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.15000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 0c7629ba750aee7d5830cdc8c9429aae1f0dffa6",
        "desc": "addr(bcrt1qp3mznwn4pth86kpsehyvjs564c0smlaxfzt480)#4p3u8rdy",
        "hex": "00140c7629ba750aee7d5830cdc8c9429aae1f0dffa6",
        "address": "bcrt1qp3mznwn4pth86kpsehyvjs564c0smlaxfzt480",
        "type": "witness_v0_keyhash"
      }
    },
    {
      "value": "0.05000000",
      "n": 1,
      "scriptPubKey": {
        "asm": "0 59e9236e250a63bb625bd493cea7090a764c6226

Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)

In [11]:
pk_hash = hash160(bytes.fromhex(alice_funding_pubkey))
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(0.201 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print("tx_digest_preimage: ",tx_digest_preimage.hex())

tx_digest_preimage:  02000000d64a5186def51ac35dfc18bf3f2179d3541027fcee8cbc7924c2f9f95dba8cb63bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044ce110bbbb6a0a698b646b62850ebebad00b5840ea4f5d65a4bf2cafe56e5b6fd000000001976a914c823d90c2b37e0bad4616612d012ede011c2fd8288aca0b3320100000000ffffffff3c01a1765aeb3e56f5f7144aea757ea975733bfb53d23fc982ebb79c2553d0290000000001000000


Now we are ready to hash this transaction and produce an ecdsa signature on it.

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [13]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + varint_len(signature)
    + signature
    + varint_len(bytes.fromhex(alice_funding_pubkey))
    + bytes.fromhex(alice_funding_pubkey)
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs # scriptsig left empty as espcified on BIP
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  02000000000101ce110bbbb6a0a698b646b62850ebebad00b5840ea4f5d65a4bf2cafe56e5b6fd0000000000ffffffff02c0e1e400000000001600140c7629ba750aee7d5830cdc8c9429aae1f0dffa6404b4c000000000022002059e9236e250a63bb625bd493cea7090a764c6226576c7a0f91a9f3d84a907dd102483045022100f32f79903eb7c743da86b1acd081e1207e30bac675ff48bd3cece0bbb25334e3022009418ae958da9422f14df3387ff01dec40098b36d5d7347c949840defeaf26ed0121028e3fe2292fbcf3d5dbe3ae8d6589022e5188e8e3045c2c817a8ef9f643246b0d00000000


### The funding_created Message
Now Alice can send Bob the [`funding_created`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-funding_created-message) message with the following information from the channel funding transaction:
- funding_txid
- funding_output_index
- signature

This message describes the outpoint which Alice has created for the initial commitment transactions.

Alice should never broadcast this funding transaction before having the Bob signature for the commitment transaction, otherwise she would have to trust Bob to get her funds back from the multisig UTXO. So let's got to the next [notebook](https://github.com/MPins/lightning-tx-tutorial/blob/master/Chapter%202%20-%20Commitment%20Transactions/Alice%20Commitment%20Transaction.ipynb) to see the commitment transactions construction in detail.

To see if our funding transaction would have been accepted, we can use the testmempoolaccept command below.


In [14]:
result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

[{'txid': '2f72dcb63ff1f44b02f0f27218f68b694787d2dc8fd8a16ced438be850c9264e', 'wtxid': '4f7c04001a3f5fb457b7c1cf07566e4680c7a538d874a8c2847e5de2fb905ef2', 'allowed': True, 'vsize': 153, 'fees': {'base': Decimal('0.00100000'), 'effective-feerate': Decimal('0.00653594'), 'effective-includes': ['4f7c04001a3f5fb457b7c1cf07566e4680c7a538d874a8c2847e5de2fb905ef2']}}]
